# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [137]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [138]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/Users/hudsonpierce/Documents/Learning/Data_Engineering/Data_Modeling/project


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [139]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [140]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [141]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [142]:
session.execute("CREATE KEYSPACE IF NOT EXISTS data_modeling_project \
WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }")

#### Set Keyspace

In [143]:
session.set_keyspace('data_modeling_project')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [144]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS songs_by_session_id_and_item_in_session "
query = query + "(session_id int, item_in_session int, artist_name varchar, song_title varchar, song_length double, PRIMARY KEY (session_id, item_in_session))"
session.execute(query)

In [145]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Assigns the INSERT statements into the `query` variable
        query = "INSERT INTO songs_by_session_id_and_item_in_session (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assigns which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [146]:
rows = session.execute("SELECT artist_name, song_title, song_length FROM songs_by_session_id_and_item_in_session WHERE session_id=338 AND item_in_session=4")
for row in rows:
    print(f"{row.artist_name}, {row.song_title}, {row.song_length}")

Faithless, Music Matters (Mark Knight Dub), 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [147]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS songs_by_user_id_and_session_id "
query = query + "(user_id int, session_id int, item_in_session int, artist_name varchar, song_title varchar, first_name varchar, last_name varchar, PRIMARY KEY((user_id, session_id), item_in_session))"
session.execute(query)

In [148]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Assigns the INSERT statements into the `query` variable
        query = "INSERT INTO songs_by_user_id_and_session_id (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assigns which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [149]:
rows = session.execute("SELECT artist_name, song_title, first_name, last_name FROM songs_by_user_id_and_session_id WHERE user_id=10 AND session_id=182")
for row in rows:
    print(f"{row.artist_name}, {row.song_title}, {row.first_name} {row.last_name}")
                    

Down To The Bone, Keep On Keepin' On, Sylvie Cruz
Three Drives, Greece 2000, Sylvie Cruz
Sebastien Tellier, Kilometer, Sylvie Cruz
Lonnie Gordon, Catch You Baby (Steve Pitron & Max Sanna Radio Edit), Sylvie Cruz


In [150]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS users_by_song_title "
query = query + "(song_title varchar, user_id int, first_name varchar, last_name varchar, PRIMARY KEY(song_title, user_id))"
session.execute(query)


In [151]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Assigns the INSERT statements into the `query` variable
        query = "INSERT INTO users_by_song_title (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assigns which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [152]:
rows = session.execute("SELECT first_name, last_name FROM users_by_song_title WHERE song_title='All Hands Against His Own'")
for row in rows:
    print(f"{row.first_name} {row.last_name}")

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [153]:
session.execute("DROP TABLE songs_by_session_id_and_item_in_session")

In [154]:
session.execute("DROP TABLE songs_by_user_id_and_session_id")

In [155]:
session.execute("DROP TABLE users_by_song_title")

### Close the session and cluster connection¶

In [156]:
session.shutdown()
cluster.shutdown()